#**Airspace SVG**


*This script will extract the relavent data from the DAFIF and create a file which the webpage can understand*

*   First **upload DAFIDUK_ED_8-1.zip** file to this page (file icon, 
left hand side) this will take a few minutes to upload.
*    From the toolbar up top select Runtime then **Run All** to run the script
*    Once the program has finished **refresh the folder** to show the airspace.svg file. 
*    **Download airspace.svg** andplace in the folder containing the airspace trainer webpage.

**YOU SHOULD NOT NEED TO CHANGE ANYTHING BELOW HERE**

##Imports, Constants & Gobal Functions

Import Libraries


In [29]:
import pandas as pd
import math
import re
from zipfile import ZipFile

Constants

In [30]:
mapWidth = 784.225 ;
mapHeight = 978.724;

mapLonLeft = -11.45;
mapLonRight = 4.30;
mapLonDelta = mapLonRight - mapLonLeft;

mapLatBottom = 49.75;
mapLatBottomDegree = mapLatBottom * math.pi / 180;

lowerRegex = 'FL([0-9][0-9]{1,2})|([0-9]{5})AMSL|([0-9]{5})AGL|([0-9]{5}$)|(SURFACE)|(GND)|(BY NOTAM)|(U)'
upperRegex = 'FL([0-9][0-9]{1,2})|([0-9]{6})AMSL|([0-9]{6})AGL|([0-9]{5}$)|(UNLTD)|(BY NOTAM)|(U)'

suasType = {"A": "ALERT", "D": "DANGER", "M": "MILITARY OPERATIONS AREA", "P": "PROHIBITED", "R": "RESTRICTED", "T": "TEMPORARY RESERVED AIRSPACE", "W": "WARNING", }

Global Functions


In [31]:
def convertGeoToPixel(lat, lon):
  x = (lon - mapLonLeft) * (mapWidth / mapLonDelta);
  lat = lat * math.pi / 180;
  worldMapWidth = ((mapWidth / mapLonDelta) * 360) / (2 * math.pi);
  mapOffsetY = (worldMapWidth / 2 * math.log((1 + math.sin(mapLatBottomDegree)) / (1 - math.sin(mapLatBottomDegree))));
  y= mapHeight - ((worldMapWidth / 2 * math.log((1 + math.sin(lat)) / (1 - math.sin(lat)))) - mapOffsetY);
  return round(x,3), round(y,3)

def convertPixelToGeo(x, y) :
  worldMapRadius = ((mapWidth / mapLonDelta) * 360) / (2 * math.pi);
  mapOffsetY = (worldMapRadius / 2) * math.log((1 + math.sin(mapLatBottomDegree)) / (1 - math.sin(mapLatBottomDegree)));
  equatorY = mapHeight + mapOffsetY;
  a = (equatorY - y) / worldMapRadius;
  lat = (180 / math.pi) * (2 * math.atan(math.exp(a)) - math.pi / 2);
  lon = mapLonLeft + (x / mapWidth) * mapLonDelta;
  return round(lat, 3), round(lon,3);

def pixelsPerNM(lat):
  return mapWidth/(mapLonDelta*(math.cos(lat*math.pi/180)*60))

def parse(rows, df, path, circle, par=""):
  if(path):
    return createPath(rows, df, par)
  elif(circle):
    return createCircle(rows, df, par)
  else:
    return createPolygon(rows, df, par)

def createArc(row):
  lat = row.WGS_DLAT0
  lon = row.WGS_DLONG0
  x0,y0 = convertGeoToPixel(lat,lon)
  r = round(row.RADIUS1 * pixelsPerNM(lat),3)

  lat = row.WGS_DLAT1
  lon = row.WGS_DLONG1
  x1,y1 = convertGeoToPixel(lat,lon)

  lat = row.WGS_DLAT2
  lon = row.WGS_DLONG2
  x2,y2 = convertGeoToPixel(lat,lon)

  sf = 0 if row.SHAP == 'L' else 1

  T = math.atan2(y0 - y1, x0 - x1) * 180/math.pi
  C = math.atan2(y2 - y1,x2 - x1) * 180/math.pi
  δ=(T-C+540)%360-180
  dir = 'L' if δ < 0 else 'R'
  laf = 0 if dir == row.SHAP else 1

  return f'A {r} {r} 0 {laf} {sf} {x2} {y2}'

def createCircle(rows, df, par):
  area = df.iloc[rows,:] 
  for row in area.itertuples():
    lat = row.WGS_DLAT0
    lon = row.WGS_DLONG0
    x,y = convertGeoToPixel(lat,lon)
    r = round(row.RADIUS1 * pixelsPerNM(lat),3)
    return f'<circle {par} id="{name}" cx="{x}" cy="{y}" r="{r}" vector-effect="non-scaling-stroke"></circle>'

def createPolygon(rows, df, par):
  points = []
  area = df.iloc[rows,:] 
  for row in area.itertuples():
    lat = row.WGS_DLAT1
    lon = row.WGS_DLONG1
    x,y = convertGeoToPixel(lat,lon)
    points.append(f'{str(x)} {str(y)}')
  points = ' '.join(points)
  return f'<polygon {par} id="{name}" points="{points}" vector-effect="non-scaling-stroke"></polygon>'

def createPath(rows, df, par):
  area = df.iloc[rows,:] 
  points = []
  for index, row in enumerate(area.itertuples()):

    if index == 0:
      lat = row.WGS_DLAT1
      lon = row.WGS_DLONG1
      x,y = convertGeoToPixel(lat,lon)    
      points.append(f'{str(x)} {str(y)}')

    if row.SHAP == 'L' or row.SHAP == 'R':
      points.append(createArc(row))
    elif index != -1:
      lat = row.WGS_DLAT2
      lon = row.WGS_DLONG2
      x,y = convertGeoToPixel(lat,lon) 
      points.append(f'L {str(x)} {str(y)}')   
  return f'<path {par} id="{name}" d="M {" ".join(points)} Z" vector-effect="non-scaling-stroke"></path>'

def bdrypar(id, df):
  row = df.loc[df['BDRY_IDENT'] == id]
  classification = row.CLASS.values[0]
  remarks = row.CLASS_EX_RMK.values[0]
  remarks = remarks.replace('&', 'AND')
  lower = row.LOWER_ALT.values[0]
  upper = row.UPPER_ALT.values[0]
  min = lowerToFL(lower)
  max = upperToFL(upper)
  return f'data-class="{classification}" data-remarks="{remarks}" data-lower="{lower}" data-upper="{upper}" data-min="{min}" data-max="{max}"'

def suaspar(id, sector, df):
  row = df.loc[(df['SUAS_IDENT'] == id) & (df['SECTOR'] == sector if sector else True)]
  airspaceType = suasType[row.TYPE.values[0]]
  lower = row.LOWER_ALT.values[0]
  upper = row.UPPER_ALT.values[0]
  min = lowerToFL(lower)
  max = upperToFL(upper)
  return f'data-type="{airspaceType}" data-lower="{lower}" data-upper="{upper}" data-min="{min}" data-max="{max}"'

def arfpar(id, df):
  row = df.loc[df['ARF_IDENT'] == id]
  lower = row.REFUEL1_ALT1.values[0]
  upper = row.REFUEL1_ALT2.values[0]
  min = lowerToFL(lower)
  max = upperToFL(upper)
  return f'data-lower="{lower}" data-upper="{upper}" data-min="{min}" data-max="{max}"'

def lowerToFL(lower: str):
  if any(string in lower for string in ["SURFACE", "GND", "BY NOTAM", "U"]):
    return 0
  else:
    match = re.search(lowerRegex, lower)
    level = int(next(filter(lambda el: el, match.groups())))
    if match.group(2) or match.group(3) or match.group(4):
      level = level/100
    return int(level)

def upperToFL(upper: str):
  if any(string in upper for string in ["UNLTD", "BY NOTAM", "U"]):
    return 660
  else:
    match = re.search(lowerRegex, upper)
    level = int(next(filter(lambda el: el, match.groups())))
    if match.group(2) or match.group(3) or match.group(4):
      level = level/100
    return int(level)

##Unzip the DAFIF

In [32]:
with ZipFile("DAFIFUK_ED_8-1.zip") as myZip:
    with myZip.open('Ed8.1_CD/DAFIF81/DAFIFT/WPT/WPT.TXT') as myFile:
        wpt = pd.read_csv(myFile, sep='\t', dtype={'WAC': 'str'})
    with myZip.open('Ed8.1_CD/DAFIF81/DAFIFT/ARPT/ARPT.TXT') as myFile:
        arpt = pd.read_csv(myFile, sep='\t', dtype={})
    with myZip.open('Ed8.1_CD/DAFIF81/DAFIFT/SUAS/SUAS.TXT') as myFile:
        suas = pd.read_csv(myFile, sep='\t', converters={
                           'SECTOR': str, 'NAME': str})
    with myZip.open('Ed8.1_CD/DAFIF81/DAFIFT/SUAS/SUAS_PAR.TXT') as myFile:
        suas_par = pd.read_csv(myFile, sep='\t')
    with myZip.open('Ed8.1_CD/DAFIF81/DAFIFT/BDRY/BDRY.TXT') as myFile:
        bdry = pd.read_csv(myFile, sep='\t', converters={'DERIVATION': str})
    with myZip.open('Ed8.1_CD/DAFIF81/DAFIFT/BDRY/BDRY_PAR.TXT') as myFile:
        bdry_par = pd.read_csv(myFile, sep='\t', converters={
                               'CLASS': str, 'CLASS_EX_RMK': str})
    with myZip.open('Ed8.1_CD/DAFIF81/DAFIFT/ARF/ARF_SEG.TXT') as myFile:
        arf = pd.read_csv(myFile, sep='\t')
    with myZip.open('Ed8.1_CD/DAFIF81/DAFIFT/ARF/ARF_PAR.TXT') as myFile:
        arf_par = pd.read_csv(myFile, sep='\t')
    with myZip.open('Ed8.1_CD/DAFIF81/DAFIFT/ATS/ATS.TXT') as myFile:
        ats = pd.read_csv(myFile, sep='\t', converters={
                          'WPT1_DESC_4': str, 'WPT2_DESC_4': str, 'OUTBD_CRS': str, 'INBD_CRS': str})
    with myZip.open('Ed8.1_CD/DAFIF81/VERSION') as myFile:
        version = myFile.readline().decode('utf-8')
        cycle = pd.to_datetime(version[:4], format="%y%m")
        cycle = f'{cycle.month_name()[:3]} {cycle.year}'
        expirationDate = pd.to_datetime(version[12:20],format="%d%m%Y")
        expirationDate =pd.Timestamp(expirationDate).timestamp()

#Airspace Definitions

##Defs & Styles

In [33]:
def createDefsAndStyles():
  airspace.write('<defs>'
    '<pattern id="aar-pattern" x="0" y="0" width="0.3" height="0.3" patternUnits="userSpaceOnUse">'
    '<circle cx="0.1" cy="0.1" r="0.1" fill="rgb(50, 46, 42)"></circle>'
    '</pattern>'
    '<polygon id="waypoint-pattern" x="-0.5" y="-0.25" width="2" height="1" stroke="rgb(80,80,80)" stroke-width="0.1" fill="transparent" points="0,-0.25 0.5,0.5 -0.5,0.5"/>'
    '<rect id="airfield-pattern" x="-0.5" y="-0.5" width="1" height="1" stroke="rgb(80,80,80)" stroke-width="0.1" fill="transparent"/>'
    '</defs>')
  airspace.write('<style>text {fill: rgb(80, 80, 80); font-family: Arial, sans-serif; font-size: 2px; white-space: pre; }</style>')

##Waypoints


In [34]:
def createWaypoints():
  airspace.write('<!-- WAYPOINTS -->')
  #Create group for all waypoints
  airspace.write('<g id="waypoints">')

  for row in wpt.itertuples():
    if row.CTRY != 'UK' or row.TYPE != 'R':
      continue
    lat = row.WGS_DLAT
    lon = row.WGS_DLONG
    x,y = convertGeoToPixel(lat,lon)

    #Element
    airspace.write('<g>')
    airspace.write(f'<use id="{row.WPT_IDENT}" x="{x}" y="{y}" href="#waypoint-pattern"/>')
    airspace.write(f'<text x="{x+1}" y="{y+1}">{row.WPT_IDENT.capitalize()}</text>')
    airspace.write('</g>')
    
  #Close group
  airspace.write('</g>\n')

##Airfields


In [35]:
def createAirfields():
  airspace.write('<!-- Airfields -->')
  #Create group for all airfields
  airspace.write('<g id="airfields">')

  for row in arpt.itertuples():
    if not row.ICAO.startswith('EG'):
      # UK only
      continue
    lat = row.WGS_DLAT
    lon = row.WGS_DLONG
    x,y = convertGeoToPixel(lat,lon)

    #Element
    airspace.write('<g>')
    airspace.write(f'<use id="{row.ICAO}" x="{x}" y="{y}" href="#airfield-pattern"/>')
    airspace.write(f'<text x="{x+1}" y="{y+1}">{row.ICAO.upper()}</text>')
    airspace.write('</g>')
    
  #Close group
  airspace.write('</g>\n')

##SUAS 



In [36]:
def whichGroup(row):
  if row.TYPE == 'D':
    if not 'Z' in row.SECTOR:
      return danger
  if row.TYPE == 'M':
    if row.NAME.startswith('TRA '):
      return tra
    if row.NAME.startswith('E3 ORBIT AREA'):
      return orbit
      
indices = []
name = None
path = False
circle = False
tra = []
orbit = []
danger = []
ident = None
sector = None

for index, row in enumerate(suas.itertuples()):
  if not row.ICAO.startswith('EG'):
    # UK only
    continue
  if not (row.TYPE == 'D' and not 'Z' in row.SECTOR)  and not (row.TYPE == 'M' and (row.NAME.startswith('TRA ') or row.NAME.startswith('E3 ORBIT AREA'))):
    continue

  if name == None:
    name = row.SUAS_IDENT + row.SECTOR;
    ident = row.SUAS_IDENT
    sector = row.SECTOR
    group = whichGroup(row)
  if name != row.SUAS_IDENT + row.SECTOR:
    group.append(parse(indices, suas, path, circle, suaspar(ident, sector, suas_par)))
    name = row.SUAS_IDENT + row.SECTOR;
    ident = row.SUAS_IDENT
    sector = row.SECTOR
    group = whichGroup(row)
    indices = []
    path = False
    circle = False
  indices.append(index)
  if row.SHAP == 'L' or row.SHAP == 'R':
    path = True
  if row.SHAP == 'C':
    circle = True
group.append(parse(indices, suas, path, circle, suaspar(ident, sector, suas_par)))

def createDangerAreas():
  airspace.write('<!-- Danger Areas -->')
  airspace.write('<g id="danger-areas">')
  airspace.write('\n'.join(danger))
  airspace.write('</g>\n')

def createTRAs():
  airspace.write('<!-- TRAs -->')
  airspace.write('<g id="tras" style="stroke: rgb(95, 94, 109); stroke-width: 0.5; fill: none;">')
  airspace.write('\n'.join(tra))
  airspace.write('</g>\n')

def createOrbits():
  airspace.write('<!-- Orbits -->')
  airspace.write('<g id="orbits" style="stroke: rgb(25, 99, 26); stroke-width: 0.5; fill: none;">')
  airspace.write('\n'.join(orbit))
  airspace.write('</g>\n')

##Boundary 



In [37]:
def whichGroup(row):
  if row.TYPE == 4:
    return ACC
  if row.TYPE == 6:
    return CTA
  if row.TYPE == 7:
    return CTZ
  if row.TYPE == 8:
    return FIR
  if row.TYPE == 9:
    return OCA
  if row.TYPE == 10:
    return Radar
  if row.TYPE == 11:
    return TCA
  if row.TYPE == 12:
    return UIR
  if row.TYPE == 14:
    return Other
    
indices = []
name = None
path = False
circle = False
ACC = []
CTA = []
CTZ = []
FIR = []
OCA = []
Radar = []
TCA = []
UIR = []
Other = []
ident = None

for index, row in enumerate(bdry.itertuples()):
  if not row.ICAO.startswith('EG'):
    # UK only
    continue
  if 'UPPER AIRSPACE' in row.NAME:
    continue

  if name == None:
    name = row.NAME;
    ident = row.BDRY_IDENT
    group = whichGroup(row)
  if name != row.NAME:
    group.append(parse(indices, bdry, path, circle, bdrypar(ident, bdry_par)))
    name = row.NAME;
    ident = row.BDRY_IDENT
    group = whichGroup(row)
    indices = []
    path = False
    circle = False
  indices.append(index)
  if row.SHAP == 'L' or row.SHAP == 'R':
    path = True
  if row.SHAP == 'C':
    circle = True
group.append(parse(indices, bdry, path, circle, bdrypar(ident, bdry_par)))

#Create ACCs
def createACCs():
  airspace.write('<!-- ACCs -->')
  airspace.write('<g id="ACCs" style="stroke: rgb(80, 80, 80); stroke-width: 0.5; stroke-opacity: 0.2; fill: none;">')
  airspace.write('\n'.join(ACC))
  airspace.write('</g>\n')

#Create CTAs
def createCTAs():
  airspace.write('<!-- CTAs -->')
  airspace.write('<g id="CTAs" style="stroke: rgb(25, 99, 26); stroke-width: 0.5; fill: rgb(25, 99, 26); fill-opacity: 0.1;">')
  airspace.write('\n'.join(CTA))
  airspace.write('</g>\n')

#Create CTZs
def createCTZs():
  airspace.write('<!-- CTZs -->')
  airspace.write('<g id="CTZs" style="stroke: rgb(25, 99, 26); stroke-width: 0.5; fill: rgb(25, 99, 26); fill-opacity: 0.1;">')
  airspace.write('\n'.join(CTZ))
  airspace.write('</g>\n')

#Create FIRs
def createFIRs():
  airspace.write('<!-- FIRs -->')
  airspace.write('<g id="FIRs" style="stroke: rgb(25, 99, 26); stroke-width: 0.5; fill: none;">')
  airspace.write('\n'.join(FIR))
  airspace.write('</g>\n')

#Create OCAs
def createOCAs():
  airspace.write('<!-- OCAs -->')
  airspace.write('<g id="OCAs" style="stroke: rgb(25, 99, 26); stroke-width: 0.5; fill: none;">')
  airspace.write('\n'.join(OCA))
  airspace.write('</g>\n')

#Create Radars
def createRadars():
  airspace.write('<!-- Radars -->')
  airspace.write('<g id="Radars" style="stroke: rgb(25, 99, 26); stroke-width: 0.5; fill: none;">')
  airspace.write('\n'.join(Radar))
  airspace.write('</g>\n')

#Create TCAs
def createTCAs():
  airspace.write('<!-- TCAs -->')
  airspace.write('<g id="TCAs" style="stroke: rgb(25, 99, 26); stroke-width: 0.5; fill: rgb(25, 99, 26); fill-opacity: 0.1;">')
  airspace.write('\n'.join(TCA))
  airspace.write('</g>\n')

#Create UIRs
def createUIRs():
  airspace.write('<!-- UIRs -->')
  airspace.write('<g id="UIRs" style="stroke: rgb(25, 99, 26); stroke-width: 0.5; fill: none;">')
  airspace.write('\n'.join(UIR))
  airspace.write('</g>\n')

#Create Others
def createOthers():
  airspace.write('<!-- Others -->')
  airspace.write('<g id="Others" style="stroke: rgb(25, 99, 26); stroke-width: 0.5; fill: none;">')
  airspace.write('\n'.join(Other))
  airspace.write('</g>\n')

##AAR

In [38]:
indices = []
name = None
path = False
circle = False
aar = []

for index,row in enumerate(arf.itertuples()):
  if not row.ICAO.startswith('EG'):
    # UK only
    continue

  if name == None:
    name = row.ARF_IDENT;
  if name != row.ARF_IDENT:
    aar.append(parse(indices, arf, path, circle, arfpar(name, arf_par)))
    indices = []
    path = False
    circle = False
    name = row.ARF_IDENT;
  indices.append(index)
  if row.SHAP == 'L' or row.SHAP == 'R':
    path = True
  if row.SHAP == 'C':
    circle = True
aar.append(parse(indices, arf, path, circle, arfpar(name, arf_par)))

def createAAR():
  airspace.write('<!-- AAR -->')
  airspace.write('<g id="aar" fill="url(#aar-pattern)">')
  airspace.write('\n'.join(aar))
  airspace.write('</g>\n')

##Air Routes

In [39]:
def createAirRoutes(): 
  points = []
  name = None
  airspace.write('<!-- Air Routes -->')
  #Create group
  airspace.write('<g id="air-routes" style="stroke: rgb(80, 80, 80); stroke-width: 0.5; stroke-opacity: 0.2; fill: none;">')

  for row in ats.itertuples():
    if not row.ICAO.startswith('EG'):
      # UK only
      continue
    if row.LEVEL == 'L':
      continue
    lat = row.WPT1_WGS_DLAT
    lon = row.WPT1_WGS_DLONG
    x,y = convertGeoToPixel(lat,lon)

    if name == None:
      name = row.ATS_IDENT;
    if name != row.ATS_IDENT:
      points = ' '.join(points)
      airspace.write(f'<polyline id="{name}" points="{points}" vector-effect="non-scaling-stroke"></polyline>')
      points = []
      name = row.ATS_IDENT;
    points.append(f'{str(x)}, {str(y)}')
  points = ' '.join(points)
  airspace.write(f'<polyline id="{name}" points="{points}" vector-effect="non-scaling-stroke"></polyline>')

  #Close group
  airspace.write('</g>\n')

#Create SVG


In [40]:
airspace = open('airspace.svg', 'w')
airspace.write(f'<?xml-stylesheet type="text/css" href="svg.css" ?> <svg viewBox="0 0 784.225 1046" xmlns="http://www.w3.org/2000/svg" data-cycle="{cycle}" data-date="{expirationDate}"><g id="matrix-group" transform="matrix(1 0 0 1 0 0)">')
createDefsAndStyles();

createAAR();

createDangerAreas();

createTRAs();

createCTAs();

createCTZs();

createTCAs();

createAirfields();

createWaypoints();

airspace.write('</g></svg>')
airspace.close()